In [53]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [54]:
mnist_dataset, mnist_info = tfds.load(
    name='mnist', 
    data_dir= 'C:\\Users\\slong\\OneDrive\\Desktop\\Python\\Data Science Bootcamp\\11-MNIST',
    with_info=True, # provides a tuple containing info about version, features, # of samples
    as_supervised = True # loads data in 2 tuple structure [input, target]
)


In [55]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64) # casts number to specified data type

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

#.batch() adds column to tensor to say how many samples to batch. Higher batch size is faster, lower batch size is more accurate
# batch loss is caluclated with an average
# You don't need to batch with validation but model only takes batches therefore the entire validation sample is the batch
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [56]:
input_size = 784
output_size = 10
hidden_layer_size = 300

#For activation functions: https://www.tensorflow.org/api_docs/python/tf/keras/activations

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape =(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose optimiser and loss funciton

In [59]:
custom_optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [60]:
NUM_EPOCHS=5

model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    validation_data =(validation_inputs,validation_targets),
    verbose=2
)


Epoch 1/5
540/540 - 4s - loss: 0.1272 - accuracy: 0.9617 - val_loss: 0.0890 - val_accuracy: 0.9738
Epoch 2/5
540/540 - 3s - loss: 0.0742 - accuracy: 0.9772 - val_loss: 0.0810 - val_accuracy: 0.9755
Epoch 3/5
540/540 - 4s - loss: 0.0536 - accuracy: 0.9830 - val_loss: 0.0583 - val_accuracy: 0.9813
Epoch 4/5
540/540 - 4s - loss: 0.0389 - accuracy: 0.9874 - val_loss: 0.0501 - val_accuracy: 0.9860
Epoch 5/5
540/540 - 4s - loss: 0.0317 - accuracy: 0.9891 - val_loss: 0.0423 - val_accuracy: 0.9872


## Test the model

In [61]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 13ms/step - loss: 0.0757 - accuracy: 0.9785


In [66]:
print(f"Test loss: {test_loss:.2f}\nTest accuracy: {test_accuracy*100:.2f}%")

Test loss: 0.08
Test accuracy: 97.85%
